### Imports

In [11]:
from openfhe import *
from src.encoder import CKKSEncoder
import numpy as np

### Define Vars

In [12]:
mult_depth = 0
scale_mod_size = 40
N = 4
batch_size = N//2

x1 = [0.25, 0.5]
x2 = [5.0, 4.0]

#### Set Developed Encoder

In [13]:
ckks_encoder = CKKSEncoder(N, scale=2**scale_mod_size)  

#### Set Real Encoder

In [14]:
parameters = CCParamsCKKSRNS()
parameters.SetSecurityLevel(HEStd_NotSet);

parameters.SetMultiplicativeDepth(mult_depth)
parameters.SetScalingModSize(scale_mod_size)
parameters.SetBatchSize(batch_size)
parameters.SetRingDim(N)

cc = GenCryptoContext(parameters)
cc.Enable(PKESchemeFeature.PKE)
cc.Enable(PKESchemeFeature.KEYSWITCH)
cc.Enable(PKESchemeFeature.LEVELEDSHE)

print("The CKKS scheme is using ring dimension: " + str(cc.GetRingDimension()))

The CKKS scheme is using ring dimension: 4


In [ ]:
from src.polynomial import PolynomialVec
from numpy.polynomial import Polynomial

vec1 = [1, 2, 3, 9]
vec2 = [4, 5, 6]

poly1 = PolynomialVec(vec1)
poly2 = PolynomialVec(vec2)
p1 = Polynomial(vec1)
p2 = Polynomial(vec2)

sum_poly = poly1 + poly2
mult_poly = poly1 * poly2

print("poly1: ", poly1, p1.coef)
print("poly2: ", poly2, p2.coef)
print("sum_poly: ", sum_poly, (p1 + p2).coef)
print("mult_poly: ", mult_poly, (p1 * p2).coef)

poly1:  [1 2 3 9]
poly2:  [4 5 6]
sum_poly:  [5 7 9 9]
mult_poly:  [ 4 13 28 63 63 54]


#### Operations

In [18]:

print("\nSimulated Implementation")

print("\nInput x1: " + str(x1))
print("Input x2: " + str(x2))

print("\n\n ----------- Check sigma Function -------------")
sig1 = ckks_encoder.sigma_inverse(np.concatenate((x1, x1)))
sig2 = ckks_encoder.sigma_inverse(np.concatenate((x2, x2)))
simSum = sig1 + sig2
simMulti = sig1 * sig2

decSig1 = ckks_encoder.sigma(sig1)
decSig2 = ckks_encoder.sigma(sig2)
decSimSum = ckks_encoder.sigma(simSum)
decSimMulti = ckks_encoder.sigma(simMulti)

print("\nEncoded Sig1: " + str(sig1))
print("Encoded Sig2: " + str(sig2))
print("Encoded Sig1 + Sig2: " + str(simSum))
print("Encoded Sig1 * Sig2: " + str(simMulti))

print("\nDecoded Sig1: " + str(decSig1))
print("Decoded Sig2: " + str(decSig2))
print("Decoded Sig1 + Sig2: " + str(decSimSum))
print("Decoded Sig1 * Sig2: " + str(decSimMulti))
print("---------------------------------------------")


print("\n\n ----------- Check Full Encoding -------------")
c1 = ckks_encoder.encode(x1)
c2 = ckks_encoder.encode(x2)
c_multi = (c1 * c2) % (2**10)
c_sum = c1 + c2

print("Encoded x1: " + str(c1))
print("Encoded x2: " + str(c2))
print("Encoded x1 + x2: " + str(c_sum))
print("Encoded x1 * x2: " + str(c_multi))

decC1 = ckks_encoder.decode(c1)
decC2 = ckks_encoder.decode(c2)
decSum = ckks_encoder.decode(c_sum)
decMulti = (ckks_encoder.decode(c_multi) / ckks_encoder.scale) 

print("\nDecoded x1: " + str(decC1))
print("Decoded x2: " + str(decC2))
print("Decoded x1 + x2: " + str(decSum))
print("Decoded x1 * x2: " + str(decMulti))





Simulated Implementation

Input x1: [0.25, 0.5]
Input x2: [5.0, 4.0]


 ----------- Check sigma Function -------------

Encoded Sig1: [ 3.75000000e-01+1.56125113e-17j -1.22663473e-18+4.04789462e-17j
 -9.88792381e-17+1.25000000e-01j -1.22663473e-18-4.04789462e-17j]
Encoded Sig2: [4.50000000e+00-6.24500451e-17j 4.90653893e-18-1.61915785e-16j
 3.95516953e-16-5.00000000e-01j 4.90653893e-18+1.61915785e-16j]
Encoded Sig1 + Sig2: [4.87500000e+00-4.68375339e-17j 3.67990420e-18-1.21436839e-16j
 2.96637714e-16-3.75000000e-01j 3.67990420e-18+1.21436839e-16j]
Encoded Sig1 * Sig2: [2.+0.j]

Decoded Sig1: [0.25-8.12812816e-18j 0.5 -3.36780300e-18j 0.25-1.39252217e-18j
 0.5 -3.16790322e-18j]
Decoded Sig2: [5.+3.25125126e-17j 4.+1.34712120e-17j 5.+5.57008866e-18j
 4.+1.26716129e-17j]
Decoded Sig1 + Sig2: [5.25+2.43809964e-17j 4.5 +1.00830802e-17j 5.25+4.18773089e-18j
 4.5 +9.47321648e-18j]
Decoded Sig1 * Sig2: [2.+0.j 2.+0.j 2.+0.j 2.+0.j]
---------------------------------------------


 ----------- 